#### **Sequence to Sequence Learning with Neural Networks (NIPS 2014)** 실습
* 본 코드는 기본적으로 **Seq2Seq** 논문의 내용을 따릅니다.
    * 본 논문은 **딥러닝 기반의 자연어 처리** 기법의 기본적인 구성을 이해하고 공부하는 데에 도움을 줍니다.
    * 2020년 기준 가장 뛰어난 번역 모델은 Seq2Seq가 아닌 **Transformer 기반의 모델**입니다.
* 코드 실행 전에 **[런타임]** → **[런타임 유형 변경]** → 유형을 **GPU**로 설정합니다.

#### **데이터 전처리(Preprocessing)**

* **spaCy 라이브러리**: 문장의 토큰화(tokenization), 태깅(tagging) 등의 전처리 기능을 위한 라이브러리
  * 영어(Engilsh)와 독일어(Deutsch) 전처리 모듈 설치

In [3]:
%%capture
!python -m spacy download en
!python -m spacy download de

In [2]:
import spacy

spacy_en = spacy.load("en_core_web_sm") # 영어 토큰화(tokenization)
spacy_de = spacy.load("de_core_news_sm") # 독일어 토큰화(tokenization)

In [3]:
# 간단히 토큰화(tokenization) 기능 써보기
tokenized = spacy_en.tokenizer("I am a graduate student.")

for i, token in enumerate(tokenized):
    print(f"인덱스 {i}: {token.text}")

인덱스 0: I
인덱스 1: am
인덱스 2: a
인덱스 3: graduate
인덱스 4: student
인덱스 5: .


* 영어(English) 및 독일어(Deutsch) **토큰화 함수** 정의

In [4]:
# 독일어(Deutsch) 문장을 토큰화한 뒤에 순서를 뒤집는 함수
def tokenize_de(text):
    return [token.text for token in spacy_de.tokenizer(text)][::-1]

# 영어(English) 문장을 토큰화 하는 함수
def tokenize_en(text):
    return [token.text for token in spacy_en.tokenizer(text)]

* **필드(field)** 라이브러리를 이용해 데이터셋에 대한 구체적인 전처리 내용을 명시합니다.
* 번역 목표
    * 소스(SRC): 독일어
    * 목표(TRG): 영어

In [10]:
import torchtext

* 대표적인 영어-독어 번역 데이터셋인 **Multi30k**를 불러옵니다.

In [1]:
!pip install torchdata

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.4 MB 27.8 MB/s 
     |████████████████████████████████| 140 kB 70.4 MB/s 
     |████████████████████████████████| 127 kB 67.6 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [5]:
import torchdata
from torchtext.datasets import Multi30k

train_dataset = Multi30k(root='.data', split=('train'), language_pair=("de", "en"))

In [6]:
en = []
de = []
for label, line in train_dataset:
    de+=[tokenize_de(label)]
    en+=[tokenize_en(line)]

In [ ]:
en[:2]

[['Two',
  'young',
  ',',
  'White',
  'males',
  'are',
  'outside',
  'near',
  'many',
  'bushes',
  '.'],
 ['Several',
  'men',
  'in',
  'hard',
  'hats',
  'are',
  'operating',
  'a',
  'giant',
  'pulley',
  'system',
  '.']]

* **필드(field)** 객체의 **build_vocab** 메서드를 이용해 영어와 독어의 단어 사전을 생성합니다.
  * **최소 2번 이상** 등장한 단어만을 선택합니다.

In [7]:
sos_token='<sos>'
eos_token='<eos>'

In [11]:
TRG=torchtext.vocab.build_vocab_from_iterator(en,min_freq=2,specials=['<pad>','<unk>',sos_token,eos_token])
SRC=torchtext.vocab.build_vocab_from_iterator(de,min_freq=2,specials=['<pad>','<unk>',sos_token,eos_token])
TRG.set_default_index(1)
SRC.set_default_index(1)

In [12]:
print(len(TRG.vocab))
print(len(SRC.vocab))

6191
8014


In [13]:
SRC[sos_token]

2

In [14]:
SRC[eos_token]

3

In [15]:
SRC["adsafwe"]

1

In [ ]:
SRC.vocab.get_stoi()

* 한 문장에 포함된 단어가 연속적으로 **LSTM**에 입력되어야 합니다.
    * 따라서 하나의 배치에 포함된 문장들이 가지는 단어의 개수가 유사하도록 만들면 좋습니다.
    * 이를 위해 BucketIterator를 사용합니다.
    * **배치 크기(batch size)**: 128

In [17]:
def collate_batch(batch):
    de_list,en_list=[],[]
    for (_de,_en) in batch:
        de_list.append(torch.tensor(SRC([sos_token]+tokenize_de(_de)+[eos_token]),dtype=torch.int64))
        en_list.append(torch.tensor(TRG([sos_token]+tokenize_en(_en)+[eos_token]),dtype=torch.int64))
    inputs=torch.nn.utils.rnn.pad_sequence(de_list+en_list, batch_first=True)
    de_inputs = inputs[:len(batch)]
    en_inputs = inputs[len(batch):]
    return (de_inputs.contiguous(),en_inputs.contiguous())

In [20]:
import torch
from torch.utils.data import DataLoader

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

BATCH_SIZE = 128

train_iterator,test_iterator,valid_iterator = iter(Multi30k(root='.data', split=('train','test','valid'), language_pair=("de", "en")))


train_dataloader = DataLoader(train_iterator, batch_size=BATCH_SIZE, shuffle=True,collate_fn=collate_batch)
test_dataloader = DataLoader(test_iterator, batch_size=BATCH_SIZE, shuffle=False,collate_fn=collate_batch)
valid_dataloader = DataLoader(valid_iterator, batch_size=BATCH_SIZE,collate_fn=collate_batch)

In [19]:
# for (i,j) in train_dataloader:
#     print(i)
#     print(i.shape)
#     print(j)
#     print(j.shape)
#     break

tensor([[   2,    4,   30,  ...,    0,    0,    0],
        [   2,    4,  653,  ...,    0,    0,    0],
        [   2,    4,  214,  ...,    0,    0,    0],
        ...,
        [   2,    4, 1779,  ...,    0,    0,    0],
        [   2,    4,   15,  ...,    0,    0,    0],
        [   2,    4, 2962,  ...,    0,    0,    0]])
torch.Size([128, 28])
tensor([[  2,   6,  16,  ...,   0,   0,   0],
        [  2,   6,  51,  ...,   0,   0,   0],
        [  2, 431,  36,  ...,   0,   0,   0],
        ...,
        [  2,   6,  12,  ...,   0,   0,   0],
        [  2,   6,  16,  ...,   0,   0,   0],
        [  2,   6,  12,  ...,   0,   0,   0]])
torch.Size([128, 28])


In [116]:
# cnt=0
# a=[]
# for i, batch in enumerate(train_iterator):
#     a.append(batch)
#     print(i)
#     print(batch)
#     cnt+=1
#     if cnt>4:
#         break

0
('Eine Kindergruppe sitzt zusammen und bastelt an etwas.', 'A group of children sit together and work on arts and crafts.')
1
('Ein asiatischer Mann sitzt auf einer Straße in der Stadt.', 'An Asian man sitting on a city street.')
2
('Ein Mann kickt einen Football, während im Hintergrund Fans der gegnerischen Mannschaft zuschauen.', 'A man punting a football as fans from the opposing team watch in the background')
3
('Ein weißer Mann mit Haaren und Bart und einem Kind auf den Schultern.', 'A white man with carrying hair and a beard with a child on his shoulders.')
4
('Ein Mann steht in einer heruntergekommen wirkenden Gegend.', 'A man stands in a run-down looking area.')


/usr/local/lib/python3.7/dist-packages/torch/utils/data/datapipes/iter/combining.py:249: UserWarning: Some child DataPipes are not exhausted when __iter__ is called. We are resetting the buffer and each child DataPipe will read from the start again.
  "the buffer and each child DataPipe will read from the start again.", UserWarning)


In [ ]:
# a[0][0]

'Musiker üben und unter Anleitung eines Dirigenten.'

In [ ]:
# a[0][1]

'Musicians are practicing as directed by a director.'

In [ ]:
# SRC(tokenize_de(a[0][0].lower()))

[3, 0, 44, 0, 129, 8, 1040, 0]

#### **인코더(Encoder) 아키텍처**

* 주어진 소스 문장을 **문맥 벡터(context vector)로 인코딩**합니다.
* LSTM은 hidden state과 cell state을 반환합니다.
* 하이퍼 파라미터(hyperparameter)
    * **input_dim**: 하나의 단어에 대한 원핫 인코딩 차원
    * **embed_dim**: 임베딩(embedding) 차원
    * **hidden_dim**: 히든 상태(hidden state) 차원
    * **n_layers**: RNN 레이어의 개수
    * **dropout_ratio**: 드롭아웃(dropout) 비율

In [39]:
import torch.nn as nn

# 인코더(Encoder) 아키텍처 정의
class Encoder(nn.Module):
    def __init__(self, input_dim, embed_dim, hidden_dim, n_layers, dropout_ratio):
        super().__init__()

        # 임베딩(embedding)은 원-핫 인코딩(one-hot encoding)을 특정 차원의 임베딩으로 매핑하는 레이어
        self.embedding = nn.Embedding(input_dim, embed_dim)

        # LSTM 레이어
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        self.rnn = nn.LSTM(embed_dim, hidden_dim, n_layers, dropout=dropout_ratio)
        
        # 드롭아웃(dropout)
        self.dropout = nn.Dropout(dropout_ratio)

    # 인코더는 소스 문장을 입력으로 받아 문맥 벡터(context vector)를 반환        
    def forward(self, src):
        # src: [단어 개수, 배치 크기]: 각 단어의 인덱스(index) 정보
        embedded = self.dropout(self.embedding(src))
        # embedded: [단어 개수, 배치 크기, 임베딩 차원]
        print(embedded)
        outputs, (hidden, cell) = self.rnn(embedded)
        # outputs: [단어 개수, 배치 크기, 히든 차원]: 현재 단어의 출력 정보
        # hidden: [레이어 개수, 배치 크기, 히든 차원]: 현재까지의 모든 단어의 정보
        # cell: [레이어 개수, 배치 크기, 히든 차원]: 현재까지의 모든 단어의 정보

        # 문맥 벡터(context vector) 반환
        return hidden, cell

#### **디코더(Decoder) 아키텍처**

* 주어진 문맥 벡터(context vector)를 **타겟 문장으로 디코딩**합니다.
* LSTM은 hidden state과 cell state을 반환합니다.
* 하이퍼 파라미터(hyperparameter)
    * **input_dim**: 하나의 단어에 대한 원핫 인코딩 차원
    * **embed_dim**: 임베딩(embedding) 차원
    * **hidden_dim**: 히든 상태(hidden state) 차원
    * **n_layers**: RNN 레이어의 개수
    * **dropout_ratio**: 드롭아웃(dropout) 비율

In [22]:
# 디코더(Decoder) 아키텍처 정의
class Decoder(nn.Module):
    def __init__(self, output_dim, embed_dim, hidden_dim, n_layers, dropout_ratio):
        super().__init__()

        # 임베딩(embedding)은 원-핫 인코딩(one-hot encoding) 말고 특정 차원의 임베딩으로 매핑하는 레이어
        self.embedding = nn.Embedding(output_dim, embed_dim)

        # LSTM 레이어
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        self.rnn = nn.LSTM(embed_dim, hidden_dim, n_layers, dropout=dropout_ratio)
        
        # FC 레이어 (인코더와 구조적으로 다른 부분)
        self.output_dim = output_dim
        self.fc_out = nn.Linear(hidden_dim, output_dim)
        
        # 드롭아웃(dropout)
        self.dropout = nn.Dropout(dropout_ratio)

    # 디코더는 현재까지 출력된 문장에 대한 정보를 입력으로 받아 타겟 문장을 반환     
    def forward(self, input, hidden, cell):
        # input: [배치 크기]: 단어의 개수는 항상 1개이도록 구현
        # hidden: [레이어 개수, 배치 크기, 히든 차원]
        # cell = context: [레이어 개수, 배치 크기, 히든 차원]
        input = input.unsqueeze(0)
        # input: [단어 개수 = 1, 배치 크기]
        
        embedded = self.dropout(self.embedding(input))
        # embedded: [단어 개수, 배치 크기, 임베딩 차원]

        output, (hidden, cell) = self.rnn(embedded, (hidden, cell))
        # output: [단어 개수 = 1, 배치 크기, 히든 차원]: 현재 단어의 출력 정보
        # hidden: [레이어 개수, 배치 크기, 히든 차원]: 현재까지의 모든 단어의 정보
        # cell: [레이어 개수, 배치 크기, 히든 차원]: 현재까지의 모든 단어의 정보

        # 단어 개수는 어차피 1개이므로 차원 제거
        prediction = self.fc_out(output.squeeze(0))
        # prediction = [배치 크기, 출력 차원]
        
        # (현재 출력 단어, 현재까지의 모든 단어의 정보, 현재까지의 모든 단어의 정보)
        return prediction, hidden, cell

#### **Seq2Seq 아키텍처**

* 앞서 정의한 인코더(encoder)와 디코더(decoder)를 가지고 있는 하나의 아키텍처입니다.
    * **인코더(encoder)**: 주어진 소스 문장을 문맥 벡터(context vector)로 인코딩합니다.
    * **디코더(decoder)**: 주어진 문맥 벡터(context vector)를 타겟 문장으로 디코딩합니다.
    * 단, **디코더는 한 단어씩** 넣어서 한 번씩 결과를 구합니다.
* **Teacher forcing**: 디코더의 예측(prediction)을 다음 입력으로 사용하지 않고, 실제 목표 출력(ground-truth)을 다음 입력으로 사용하는 기법

In [23]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()

        self.encoder = encoder
        self.decoder = decoder
        self.device = device

    # 학습할 때는 완전한 형태의 소스 문장, 타겟 문장, teacher_forcing_ratio를 넣기
    def forward(self, src, trg, teacher_forcing_ratio=0.5):
        # src: [단어 개수, 배치 크기]
        # trg: [단어 개수, 배치 크기]
        # 먼저 인코더를 거쳐 문맥 벡터(context vector)를 추출
        hidden, cell = self.encoder(src)

        # 디코더(decoder)의 최종 결과를 담을 텐서 객체 만들기
        trg_len = trg.shape[0] # 단어 개수
        batch_size = trg.shape[1] # 배치 크기
        trg_vocab_size = self.decoder.output_dim # 출력 차원
        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)

        # 첫 번째 입력은 항상 <sos> 토큰
        input = trg[0, :]

        # 타겟 단어의 개수만큼 반복하여 디코더에 포워딩(forwarding)
        for t in range(1, trg_len):
            output, hidden, cell = self.decoder(input, hidden, cell)

            outputs[t] = output # FC를 거쳐서 나온 현재의 출력 단어 정보
            top1 = output.argmax(1) # 가장 확률이 높은 단어의 인덱스 추출

            # teacher_forcing_ratio: 학습할 때 실제 목표 출력(ground-truth)을 사용하는 비율
            teacher_force = random.random() < teacher_forcing_ratio
            input = trg[t] if teacher_force else top1 # 현재의 출력 결과를 다음 입력에서 넣기
        
        return outputs

#### **학습(Training)**

* 하이퍼 파라미터 설정 및 모델 초기화

In [24]:
INPUT_DIM = len(SRC.vocab)
OUTPUT_DIM = len(TRG.vocab)
ENCODER_EMBED_DIM = 256
DECODER_EMBED_DIM = 256
HIDDEN_DIM = 512
N_LAYERS = 2
ENC_DROPOUT_RATIO = 0.5
DEC_DROPOUT_RATIO = 0.5

In [25]:
# 인코더(encoder)와 디코더(decoder) 객체 선언
enc = Encoder(INPUT_DIM, ENCODER_EMBED_DIM, HIDDEN_DIM, N_LAYERS, ENC_DROPOUT_RATIO)
dec = Decoder(OUTPUT_DIM, DECODER_EMBED_DIM, HIDDEN_DIM, N_LAYERS, DEC_DROPOUT_RATIO)

# Seq2Seq 객체 선언
model = Seq2Seq(enc, dec, device).to(device)

* 논문의 내용대로 $\mathcal{U}(-0.08, 0.08)$의 값으로 **모델 가중치 파라미터 초기화**

In [26]:
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.uniform_(param.data, -0.08, 0.08)
        
model.apply(init_weights)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(8014, 256)
    (rnn): LSTM(256, 512, num_layers=2, dropout=0.5)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Decoder(
    (embedding): Embedding(6191, 256)
    (rnn): LSTM(256, 512, num_layers=2, dropout=0.5)
    (fc_out): Linear(in_features=512, out_features=6191, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

* 학습 및 평가 함수 정의

In [28]:
import torch.optim as optim

# Adam optimizer로 학습 최적화
optimizer = optim.Adam(model.parameters())

# 뒷 부분의 패딩(padding)에 대해서는 값 무시
TRG_PAD_IDX = TRG["<pad>"]
criterion = nn.CrossEntropyLoss(ignore_index=TRG_PAD_IDX)

In [29]:
# 모델 학습(train) 함수
def train(model, iterator, optimizer, criterion, clip):
    model.train() # 학습 모드
    epoch_loss = 0
    count=0
    # 전체 학습 데이터를 확인하며
    for src,trg in iterator:
        src = src.to(device)
        trg = trg.to(device)
        count+=src.shape[0]
        optimizer.zero_grad()

        output = model(src, trg)
        # output: [출력 단어 개수, 배치 크기, 출력 차원]
        output_dim = output.shape[-1]
        
        # 출력 단어의 인덱스 0은 사용하지 않음
        output = output[1:].view(-1, output_dim)
        # output = [(출력 단어의 개수 - 1) * batch size, output dim]
        trg = trg[1:].view(-1)
        # trg = [(타겟 단어의 개수 - 1) * batch size]
        
        # 모델의 출력 결과와 타겟 문장을 비교하여 손실 계산
        loss = criterion(output, trg)
        loss.backward() # 기울기(gradient) 계산
        
        # 기울기(gradient) clipping 진행
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        # 파라미터 업데이트
        optimizer.step()
        
        # 전체 손실 값 계산
        epoch_loss += loss.item()
        
    
    return epoch_loss / count

In [30]:
# 모델 평가(evaluate) 함수
def evaluate(model, iterator, criterion):
    model.eval() # 평가 모드
    epoch_loss = 0
    count=0
    with torch.no_grad():
        # 전체 평가 데이터를 확인하며
        for src,trg in iterator:
            src = src.to(device)
            trg = trg.to(device)
            
            count+=src.shape[0]
            # 평가할 때 teacher forcing는 사용하지 않음
            output = model(src, trg, 0)
            # output: [출력 단어 개수, 배치 크기, 출력 차원]
            output_dim = output.shape[-1]
            
            # 출력 단어의 인덱스 0은 사용하지 않음
            output = output[1:].view(-1, output_dim)
            # output = [(출력 단어의 개수 - 1) * batch size, output dim]
            trg = trg[1:].view(-1)
            # trg = [(타겟 단어의 개수 - 1) * batch size]

            # 모델의 출력 결과와 타겟 문장을 비교하여 손실 계산
            loss = criterion(output, trg)

            # 전체 손실 값 계산
            epoch_loss += loss.item()
        
    return epoch_loss / count

* 학습(training) 및 검증(validation) 진행
    * **학습 횟수(epoch)**: 20

In [31]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [32]:
import time
import math
import random

N_EPOCHS = 20
CLIP = 1
best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    start_time = time.time() # 시작 시간 기록

    train_loss = train(model, train_dataloader, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_dataloader, criterion)

    end_time = time.time() # 종료 시간 기록
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'seq2seq.pt')

    print(f'Epoch: {epoch + 1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):.3f}')
    print(f'\tValidation Loss: {valid_loss:.3f} | Validation PPL: {math.exp(valid_loss):.3f}')

Epoch: 01 | Time: 1m 28s
	Train Loss: 0.041 | Train PPL: 1.042
	Validation Loss: 0.037 | Validation PPL: 1.038


/usr/local/lib/python3.7/dist-packages/torch/utils/data/datapipes/iter/combining.py:249: UserWarning: Some child DataPipes are not exhausted when __iter__ is called. We are resetting the buffer and each child DataPipe will read from the start again.
  "the buffer and each child DataPipe will read from the start again.", UserWarning)


Epoch: 02 | Time: 1m 27s
	Train Loss: 0.037 | Train PPL: 1.038
	Validation Loss: 0.039 | Validation PPL: 1.039
Epoch: 03 | Time: 1m 27s
	Train Loss: 0.037 | Train PPL: 1.037
	Validation Loss: 0.037 | Validation PPL: 1.038
Epoch: 04 | Time: 1m 28s
	Train Loss: 0.036 | Train PPL: 1.037
	Validation Loss: 0.037 | Validation PPL: 1.037
Epoch: 05 | Time: 1m 27s
	Train Loss: 0.036 | Train PPL: 1.037
	Validation Loss: 0.037 | Validation PPL: 1.037
Epoch: 06 | Time: 1m 27s
	Train Loss: 0.036 | Train PPL: 1.037
	Validation Loss: 0.037 | Validation PPL: 1.037
Epoch: 07 | Time: 1m 28s
	Train Loss: 0.036 | Train PPL: 1.037
	Validation Loss: 0.036 | Validation PPL: 1.037
Epoch: 08 | Time: 1m 27s
	Train Loss: 0.036 | Train PPL: 1.037
	Validation Loss: 0.036 | Validation PPL: 1.037
Epoch: 09 | Time: 1m 27s
	Train Loss: 0.036 | Train PPL: 1.037
	Validation Loss: 0.036 | Validation PPL: 1.037
Epoch: 10 | Time: 1m 27s
	Train Loss: 0.036 | Train PPL: 1.037
	Validation Loss: 0.037 | Validation PPL: 1.037
E

In [80]:
# 학습된 모델 저장
from google.colab import files

files.download('seq2seq.pt')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

#### **모델 최종 테스트(testing) 결과 확인**

In [ ]:
!wget https://postechackr-my.sharepoint.com/:u:/g/personal/dongbinna_postech_ac_kr/ERgwTMYWR7FMhApROaNvZREBTjEDi00ttSzt8ZNj1PS_5g?download=1 -O seq2seq.pt

In [33]:
model.load_state_dict(torch.load('seq2seq.pt'))

test_loss = evaluate(model, test_dataloader, criterion)

print(f'Test Loss: {test_loss:.3f} | Test PPL: {math.exp(test_loss):.3f}')

Test Loss: 0.037 | Test PPL: 1.038


#### **나만의 데이터로 모델 사용해보기**

In [34]:
# 번역(translation) 함수
def translate_sentence(sentence, src_field, trg_field, model, device, max_len=50):
    model.eval() # 평가 모드

    if isinstance(sentence, str):
        nlp = spacy.load("de_core_news_sm")
        tokens = [token.text.lower() for token in nlp(sentence)]
    else:
        tokens = [token.lower() for token in sentence]

    # 처음에 <sos> 토큰, 마지막에 <eos> 토큰 붙이기
    tokens = [sos_token] +tokens + [eos_token]
    print(f"전체 소스 토큰: {tokens}")

    src_indexes = src_field(tokens)
    print(f"소스 문장 인덱스: {src_indexes}")

    src_tensor = torch.LongTensor(src_indexes).unsqueeze(1).to(device)

    # 인코더(endocer)에 소스 문장을 넣어 문맥 벡터(context vector) 계산
    with torch.no_grad():
        hidden, cell = model.encoder(src_tensor)

    # 처음에는 <sos> 토큰 하나만 가지고 있도록 하기
    trg_indexes = [trg_field[sos_token]]

    for i in range(max_len):
        # 이전에 출력한 단어가 현재 단어로 입력될 수 있도록
        trg_tensor = torch.LongTensor([trg_indexes[-1]]).to(device)

        with torch.no_grad():
            output, hidden, cell = model.decoder(trg_tensor, hidden, cell)

        pred_token = output.argmax(1).item()
        trg_indexes.append(pred_token) # 출력 문장에 더하기

        # <eos>를 만나는 순간 끝
        if pred_token == trg_field[eos_token]:
            break

    # 각 출력 단어 인덱스를 실제 단어로 변환
    trg_tokens = trg_field.lookup_tokens(trg_indexes)

    # 첫 번째 <sos>는 제외하고 출력 문장 반환
    return trg_tokens[1:]

In [38]:
example_idx = 102

train_iterator,test_iterator,valid_iterator = iter(Multi30k(root='.data', split=('train','test','valid'), language_pair=("de", "en")))
cnt=0
for src,trg in valid_iterator:
    cnt+=1
    if cnt>example_idx:
        break


print(f'소스 문장: {src}')
print(f'타겟 문장: {trg}')
print("모델 출력 결과:", " ".join(translate_sentence(src, SRC, TRG, model, device)))

소스 문장: Mehrere Menschen stehen in der Dämmerung in der Nähe einiger Bäume.
타겟 문장: Several people are standing near trees at dusk.
전체 소스 토큰: ['<sos>', 'mehrere', 'menschen', 'stehen', 'in', 'der', 'dämmerung', 'in', 'der', 'nähe', 'einiger', 'bäume', '.', '<eos>']
소스 문장 인덱스: [2, 412, 1, 53, 7, 16, 1, 7, 16, 1, 2450, 1, 4, 3]
모델 출력 결과: a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a


In [36]:
src = tokenize_de("Guten Abend.")

print(f'소스 문장: {src}')
print("모델 출력 결과:", " ".join(translate_sentence(src, SRC, TRG, model, device)))

소스 문장: ['.', 'Abend', 'Guten']
전체 소스 토큰: ['<sos>', '.', 'abend', 'guten', '<eos>']
소스 문장 인덱스: [2, 4, 1, 4172, 3]
모델 출력 결과: a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a
